In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, datetime
import numpy as np
import pandas  as pd
import logging
import json


def daily_crawler(date):

    IS_ON = True
    d_ = date.strftime("%Y%m%d")

    TODAY_DATA = False

    keys = {'syear': date.year, 'smonth': date.month, 'sday': date.day}
    try:
        r = requests.post("http://www.taifex.com.tw/chinese/3/3_1_1.asp", data = keys)
        soup = BeautifulSoup(r.text, "lxml")
        soup = soup.select('table')[2].select('table')[1].select('td')
        o = int(soup[2].text)
        h = int(soup[3].text)
        l = int(soup[4].text)
        try:
            s = int(soup[11].text)
        except:
            s = int(soup[5].text)
        v = int(soup[10].text)
        c = int(soup[5].text)
        
        r = requests.get("http://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json&date="+d_)
        r = BeautifulSoup(r.text, "lxml").select('p')[0].text
        r = json.loads(r)
        for data in r["data"]:
            if data[0].replace("/","")[-4:] == d_[-4:]:
                so = float(data[1].replace(",",""))
                sh = float(data[2].replace(",",""))
                sl = float(data[3].replace(",",""))
                sc = float(data[4].replace(",",""))

        TODAY_DATA = [date, o, h, l, s, v, c, so, sh, sl, sc]
    except Exception as e:
        print(e)
        IS_ON = False
        logger_crawler.info("Stock is not open on {}.".format(date))

    if IS_ON:
        try:
            r = requests.get("http://www.tse.com.tw/exchangeReport/FMTQIK?response=json&date="+d_)
            r = BeautifulSoup(r.text, "lxml").select('p')[0].text
            r = json.loads(r)
            for data in r["data"]:
                if data[0].replace("/","")[-4:] == d_[-4:]:
                    sv = int(data[2].replace(",",""))
                    TODAY_DATA.append(sv)
        except Exception as e:
            if IS_ON:
                logger_crawler.error("[insert_volume][{}] {}".format(date, e))

        try:
            r = requests.post("http://www.taifex.com.tw/chinese/3/7_12_3.asp", data = keys)
            r = BeautifulSoup(r.text, "lxml")

            cols_f = "f_long, f_long_contract_V, f_short, f_short_contract_V, f_acu_net, f_acu_contract_V".replace(" ","").split(",")
            cols_i = "i_long, i_long_contract_V, i_short, i_short_contract_V, i_acu_net, i_acu_contract_V".replace(" ","").split(",")
            cols_s = "d_long, d_long_contract_V, d_short, d_short_contract_V, d_acu_net, d_acu_contract_V".replace(" ","").split(",")

            for n in [5,4,3]:
                r_ = r.select('table')[2].select('tr')[1].select('table')[0].select('tr')[n].select('td')
                if n == 3:
                    l = float(r_[3].text.replace(",",""))
                    lv = float(r_[4].text.replace(",",""))
                    s = float(r_[5].text.replace(",",""))
                    sv = float(r_[6].text.replace(",",""))
                    a = float(r_[13].text.replace(",",""))
                    av = float(r_[14].text.replace(",",""))
                else:
                    l = float(r_[1].text.replace(",",""))
                    lv = float(r_[2].text.replace(",",""))
                    s = float(r_[3].text.replace(",",""))
                    sv = float(r_[4].text.replace(",",""))
                    a = float(r_[11].text.replace(",",""))
                    av = float(r_[12].text.replace(",",""))
                TODAY_DATA += [l,lv,s,sv,a,av]

        except Exception as e:
            if IS_ON:
                logger_crawler.error("[insert_3_position][{}] {}".format(date, e))
        try:
            r = requests.get("http://www.tse.com.tw/fund/BFI82U?response=json&dayDate="+d_)
            r = BeautifulSoup(r.text, "lxml").select("p")[0].text
            r = json.loads(r)
            r = r["data"]

            dsb = int(r[0][1].replace(",",""))
            dss = int(r[0][2].replace(",",""))
            dhb = int(r[1][1].replace(",",""))
            dhs = int(r[1][2].replace(",",""))
            ib = int(r[2][1].replace(",",""))
            is_ = int(r[2][2].replace(",",""))
            fb = int(r[3][1].replace(",",""))
            fs = int(r[3][2].replace(",",""))

            value = [dsb, dss, dhb, dhs, ib, is_, fb, fs]
            cols = ["d_self_buy","d_self_sell","d_hedge_buy","d_hedge_sell","i_buy","i_sell","f_buy","f_sell"]
            TODAY_DATA += value

        except Exception as e:
            if IS_ON:
                logger_crawler.error("[insert_3_BS][{}] {}".format(date, e))
    return TODAY_DATA


if __name__ == "__main__":
    log_path = "my.log"
    logging.basicConfig(level=logging.INFO, format='[%(name)s][%(levelname)s] %(asctime)s - %(message)s', filename=log_path)
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('[%(levelname)-5s] %(name)-10s: %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)
    logger_crawler = logging.getLogger('CRAWLER')

    df = pd.read_csv("dailyinfo.csv")
    cols = df.columns
    df.set_index('date_', inplace=True)

    LAST_UPDATE = datetime.strptime(df.index[-1], '%Y-%m-%d')
    LAST_UPDATE_ = LAST_UPDATE + timedelta(days=1)
    TODAY = datetime.today().strftime("%Y-%m-%d")

    logger_crawler.info("LAST UPDATE: {}".format(LAST_UPDATE))
    logger_crawler.info("TODAY IS: {}".format(TODAY))

    UPDATE_DATA = []

    for date in pd.date_range(start=LAST_UPDATE_, end=TODAY):
        date = date.date()
        DATA = daily_crawler(date)
        if DATA:
            UPDATE_DATA.append(DATA)

    df_crawl = pd.DataFrame(UPDATE_DATA, columns=cols)
    df_crawl.set_index('date_', inplace=True)
    df = df.append(df_crawl)
    df.to_csv('dailyinfo.csv')

[INFO ] CRAWLER   : LAST UPDATE: 2017-11-17 00:00:00
[INFO ] CRAWLER   : LAST UPDATE: 2017-11-17 00:00:00
[INFO ] CRAWLER   : LAST UPDATE: 2017-11-17 00:00:00
[INFO ] CRAWLER   : LAST UPDATE: 2017-11-17 00:00:00
[INFO ] CRAWLER   : TODAY IS: 2018-09-25
[INFO ] CRAWLER   : TODAY IS: 2018-09-25
[INFO ] CRAWLER   : TODAY IS: 2018-09-25
[INFO ] CRAWLER   : TODAY IS: 2018-09-25
[INFO ] CRAWLER   : Stock is not open on 2017-11-18.
[INFO ] CRAWLER   : Stock is not open on 2017-11-18.
[INFO ] CRAWLER   : Stock is not open on 2017-11-18.
[INFO ] CRAWLER   : Stock is not open on 2017-11-18.


list index out of range
list index out of range

[INFO ] CRAWLER   : Stock is not open on 2017-11-19.
[INFO ] CRAWLER   : Stock is not open on 2017-11-19.
[INFO ] CRAWLER   : Stock is not open on 2017-11-19.
[INFO ] CRAWLER   : Stock is not open on 2017-11-19.
[INFO ] CRAWLER   : Stock is not open on 2017-11-25.
[INFO ] CRAWLER   : Stock is not open on 2017-11-25.
[INFO ] CRAWLER   : Stock is not open on 2017-11-25.
[INFO ] CRAWLER   : Stock is not open on 2017-11-25.
[INFO ] CRAWLER   : Stock is not open on 2017-11-26.
[INFO ] CRAWLER   : Stock is not open on 2017-11-26.
[INFO ] CRAWLER   : Stock is not open on 2017-11-26.
[INFO ] CRAWLER   : Stock is not open on 2017-11-26.



list index out of range
list index out of range


[ERROR] CRAWLER   : [insert_volume][2017-11-30] list index out of range
[ERROR] CRAWLER   : [insert_volume][2017-11-30] list index out of range
[ERROR] CRAWLER   : [insert_volume][2017-11-30] list index out of range
[ERROR] CRAWLER   : [insert_volume][2017-11-30] list index out of range
[ERROR] CRAWLER   : [insert_3_BS][2017-11-30] HTTPConnectionPool(host='www.tse.com.tw', port=80): Max retries exceeded with url: /fund/BFI82U?response=json&dayDate=20171130 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000001D991EA3BE0>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。',))
[ERROR] CRAWLER   : [insert_3_BS][2017-11-30] HTTPConnectionPool(host='www.tse.com.tw', port=80): Max retries exceeded with url: /fund/BFI82U?response=json&dayDate=20171130 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000001D991EA3BE0>: Failed to establish a new connection: [WinError 10061] 無法連線，

...

list index out of range
list index out of range


[INFO ] CRAWLER   : Stock is not open on 2018-09-24.
[INFO ] CRAWLER   : Stock is not open on 2018-09-24.
[INFO ] CRAWLER   : Stock is not open on 2018-09-24.
[INFO ] CRAWLER   : Stock is not open on 2018-09-24.


list index out of range


[INFO ] CRAWLER   : Stock is not open on 2018-09-25.
[INFO ] CRAWLER   : Stock is not open on 2018-09-25.
[INFO ] CRAWLER   : Stock is not open on 2018-09-25.
[INFO ] CRAWLER   : Stock is not open on 2018-09-25.


invalid literal for int() with base 10: '-'


In [7]:
df.tail()

,open,high,low,settlement,volume,close,spot_open,spot_high,spot_low,spot_close,...,d_acu_net,d_acu_contract_V,d_self_buy,d_self_sell,d_hedge_buy,d_hedge_sell,i_buy,i_sell,f_buy,f_sell
date_,,,,,,,,,,,,,,,,,,,,,
2017-11-24,10841,10856,10829,10854,74534,10856,10847.79,10873.91,10833.78,10854.09,...,1915.0,4162532.0,1.107775e+09,1.286755e+09,7.382176e+09,8.129547e+09,1.264220e+09,1.474573e+09,2.477212e+10,2.783171e+10
2017-11-27,10841,10848,10737,10749,141044,10750,10836.82,10856.00,10746.74,10750.93,...,811.0,1758012.0,1.077178e+09,1.684638e+09,6.616780e+09,7.923582e+09,5.968404e+08,1.217935e+09,2.641520e+10,3.470374e+10
2017-11-28,10735,10753,10695,10708,139887,10708,10733.56,10752.58,10692.53,10707.07,...,-3019.0,-6449525.0,1.201271e+09,1.302820e+09,6.745278e+09,6.985024e+09,9.238394e+08,2.385367e+09,3.502924e+10,4.361888e+10
2017-11-29,10747,10763,10708,10728,122492,10729,10735.24,10762.91,10697.62,10713.55,...,-1367.0,-2917835.0,1.046859e+09,9.919257e+08,6.229779e+09,6.519366e+09,1.213856e+09,1.419631e+09,2.872184e+10,3.540241e+10
2017-11-30,10650,10658,10569,10576,197751,10574,10623.20,10650.18,10560.44,10560.44,...,-2892907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
